<a href="https://colab.research.google.com/github/kywch/geo-colab/blob/master/Generate_Chicago_Traces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary packages
import os, os.path
from glob import glob
from shutil import copyfile

import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
files = glob("drive/My Drive/SatTemp/chicago_synthpop_V2/*.csv.gz")
print(files)

['drive/My Drive/SatTemp/chicago_synthpop_V2/persons_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2_256p_lb_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/activities_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/places_60615.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/persons_60615.csv.gz']


In [4]:
!rm -rf synthpop
if not os.path.exists('synthpop'):
  os.mkdir('synthpop')

In [5]:
# copy these files from Google Drive to colab disk
for f in files:
  print(f)
  copyfile(f, 'synthpop/' + os.path.basename(f))

drive/My Drive/SatTemp/chicago_synthpop_V2/persons_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2_256p_lb_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/activities_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/places_60615.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/persons_60615.csv.gz


In [6]:
# look at people
persons_df = pd.read_csv('synthpop/persons_v2.csv.gz')
persons_df.info()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927761 entries, 0 to 2927760
Data columns (total 23 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   numeric_id                int64  
 1   person_id                 object 
 2   hh_id                     float64
 3   relate                    float64
 4   sex                       int64  
 5   age                       int64  
 6   gq_id                     float64
 7   school_id                 float64
 8   work_id                   float64
 9   daycare_id                float64
 10  gym_id                    float64
 11  hospital_id               float64
 12  jail                      float64
 13  jail_id                   int64  
 14  rank                      int64  
 15  weekday_schedule_id_list  object 
 16  weekend_schedule_id_list  object 
 17  otherhh_id                object 
 18  home_restaurant_id        object 
 19  recreation_id             object 
 20  grocery_id              

In [7]:
# places
places_df = pd.read_csv('synthpop/places_v2.csv.gz')
places_df.info()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210722 entries, 0 to 1210721
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   numeric_id  1210722 non-null  float64
 1   place_id    1210722 non-null  object 
 2   place_type  1210722 non-null  object 
 3   name        1093276 non-null  object 
 4   persons     1077336 non-null  float64
 5   latitude    1210668 non-null  float64
 6   longitude   1210668 non-null  float64
 7   zipcode     1204493 non-null  float64
 8   zip_grp     1194456 non-null  float64
 9   rank        1210722 non-null  int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 92.4+ MB


In [8]:
# activities
activity_df = pd.read_csv('synthpop/activities_v2.csv.gz')
activity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239781 entries, 0 to 239780
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Act_SlNo        239781 non-null  int64 
 1   start_time      239781 non-null  int64 
 2   stop_time       239781 non-null  int64 
 3   Social_Act_Loc  239781 non-null  object
 4   schedule_id     239781 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 9.1+ MB


In [13]:
# check the number of unique persons
print(len(persons_df.person_id), len(persons_df.person_id.unique()))
persons_df.head()

2927761 2927761


,numeric_id,person_id,hh_id,relate,sex,age,gq_id,school_id,work_id,daycare_id,gym_id,hospital_id,jail,jail_id,rank,weekday_schedule_id_list,weekend_schedule_id_list,otherhh_id,home_restaurant_id,recreation_id,grocery_id,worship_id,work_restaurant_id
0,1,1595930,1181524.0,1.0,2,82,NaN,NaN,NaN,579139.0,1181263.0,70.0,0.000002,1194221,100,5969|5974|5983|5911|5963|5943|5953|5898|5930|5971,12237|12229|12172|12145|12178|12207|12203|1216...,NaN,1197516|1200938|1195562|1196781|1195562,1205986|1206926|1206147|1207623|1207411,1210303|1210303|1210303,1208521.0,1196871|1195562|1200643|1200938|1195361
1,28,1597745,1181535.0,1.0,2,82,NaN,NaN,NaN,598594.0,1181263.0,70.0,0.000002,1194221,135,5914|5905|5908|5903|5921|5911|5930|5985|5965|5924,12207|12158|12180|12219|12172|12168|12216|1215...,NaN,1198007|1202973|1201510|1202764|1201711,1204187|1204188|1207623|1204185|1205247,1210303|1210303|1210271,1208491.0,1199052|1201611|1196871|1201222|1200938
2,335,1830168,1181651.0,2.0,2,80,NaN,NaN,NaN,579512.0,1181504.0,67.0,0.000075,1194221,194,5904|5963|5898|5921|5933|5902|5962|5965|5960|5903,12179|12200|12180|12152|12214|12216|12186|1214...,1106212,1198239|1199738|1201342|1195604|1203026,1206581|1204593|1208218|1204610|1206073,1210268|1210416|1210263,1209887.0,1195893|1203057|1200382|1195061|1199562
3,462,1833607,1181696.0,7.0,2,82,NaN,NaN,NaN,579254.0,1181263.0,70.0,0.000002,1194221,242,5941|5914|5902|5957|5931|5898|5985|5920|5981|5963,12229|12232|12191|12186|12226|12200|12152|1214...,NaN,1199128|1200185|1195781|1196368|1199809,1204184|1206557|1206926|1204185|1205727,1210303|1210271|1210303,1209989.0,1201510|1196631|1194644|1201711|1201711
4,656,1907903,1181820.0,1.0,2,80,NaN,NaN,NaN,598172.0,1181374.0,26.0,0.066281,1194221,84,5898|5929|5930|5913|5982|5924|5915|5903|5916|5900,12185|12193|12189|12178|12144|12152|12206|1216...,NaN,1202359|1198950|1202748|1197743|1194509,1206868|1203252|1206596|1206316|1205545,1210457|1210489|1210268,1208383.0,1201032|1195967|1198994|1199567|1202452


# Things to do
* join zip code to person -- sample 1000 each?
* for each person
  * grab the activity schedule
  * link place coordinates
  * make a row for each hour: 24 rows with coordinates
  * the coordinates for 24 hours will be used to query temperature (or heat stress)


  

In [20]:
row = next(persons_df.iterrows())[1]
print(row)

numeric_id                                                                  1
person_id                                                             1595930
hh_id                                                             1.18152e+06
relate                                                                      1
sex                                                                         2
age                                                                        82
gq_id                                                                     NaN
school_id                                                                 NaN
work_id                                                                   NaN
daycare_id                                                             579139
gym_id                                                            1.18126e+06
hospital_id                                                                70
jail                                                            

In [23]:
row.hh_id

1181524.0

In [27]:
places_df[places_df['numeric_id'] == row.hh_id]

,numeric_id,place_id,place_type,name,persons,latitude,longitude,zipcode,zip_grp,rank
1181523,1181524.0,592029,Household,hh592029,2.0,42.018917,-87.807619,60631.0,1.0,100


In [28]:
places_df[places_df['numeric_id'] == row.daycare_id]

,numeric_id,place_id,place_type,name,persons,latitude,longitude,zipcode,zip_grp,rank
579138,579139.0,170319101317,School,Saint John Lutheran School,33.0,42.016015,-87.804885,60714.0,0.0,127


In [29]:
places_df[places_df['numeric_id'] == 1210303]

,numeric_id,place_id,place_type,name,persons,latitude,longitude,zipcode,zip_grp,rank
1210303,1210303.0,967934,Supermarket,Happy Foods,NaN,42.004586,-87.818446,60631.0,NaN,0


In [33]:
test = activity_df[activity_df['schedule_id'] == 5963]
print(test)

        Act_SlNo  start_time  stop_time    Social_Act_Loc  schedule_id
132297         1           0        465         Household         5963
132298         2         465        540         Household         5963
132299         3         540        570         Household         5963
132300         4         570        590         Household         5963
132301         5         590        595         Household         5963
132302         6         595        605             Other         5963
132303         7         605        606             Other         5963
132304         8         606        610     Grocery store         5963
132305         9         610        612     Grocery store         5963
132306        10         612        617     Grocery store         5963
132307        11         617        632  Place of worship         5963
132308        12         632        647  Place of worship         5963
132309        13         647        662  Place of worship         5963
132310

In [56]:
# hour, weight, social_act_loc, coordinates (lat/lng)

curr_hour = 0

res_min = 0
prev_activity = test.Social_Act_Loc.iloc[0]
#print(curr_activity)

schedule = []

# assume that every activity starts at 0
proc_time = 0

for ii in range(test.shape[0]):
  curr_hour = test.iloc[ii].start_time // 60 
  proc_time = test.iloc[ii].start_time
  #print('start: ', curr_hour, proc_time, ii)  

  if 60*(curr_hour+1) < test.iloc[ii].stop_time:
      schedule.append([curr_hour, 
                       (60*(curr_hour+1) - test.iloc[ii].start_time)/60, 
                       test.iloc[ii].Social_Act_Loc])
      curr_hour += 1
      proc_time = 60*curr_hour

  while proc_time < test.iloc[ii].stop_time:
    if (proc_time + 60) < test.iloc[ii].stop_time:
      schedule.append([curr_hour, 1, test.iloc[ii].Social_Act_Loc])
      proc_time += 60
      curr_hour += 1
    else:
      res_act = test.iloc[ii].stop_time - proc_time
      schedule.append([curr_hour, res_act/60, test.iloc[ii].Social_Act_Loc])
      proc_time = test.iloc[ii].stop_time
  
  #print('end: ', curr_hour, proc_time, ii)      

# cleaning   

#print(schedule)





start:  0 0 0
end:  7 465 0
start:  7 465 1
end:  8 540 1
start:  9 540 2
end:  9 570 2
start:  9 570 3
end:  9 590 3
start:  9 590 4
end:  9 595 4
start:  9 595 5
end:  10 605 5
start:  10 605 6
end:  10 606 6
start:  10 606 7
end:  10 610 7
start:  10 610 8
end:  10 612 8
start:  10 612 9
end:  10 617 9
start:  10 617 10
end:  10 632 10
start:  10 632 11
end:  10 647 11
start:  10 647 12
end:  11 662 12
start:  11 662 13
end:  11 677 13
start:  11 677 14
end:  11 679 14
start:  11 679 15
end:  11 681 15
start:  11 681 16
end:  11 683 16
start:  11 683 17
end:  11 688 17
start:  11 688 18
end:  11 689 18
start:  11 689 19
end:  11 699 19
start:  11 699 20
end:  11 704 20
start:  11 704 21
end:  12 724 21
start:  12 724 22
end:  12 729 22
start:  12 729 23
end:  12 749 23
start:  12 749 24
end:  12 779 24
start:  12 779 25
end:  14 899 25
start:  14 899 26
end:  15 944 26
start:  15 944 27
end:  16 974 27
start:  16 974 28
end:  16 989 28
start:  16 989 29
end:  16 1019 29
start:  16 1

In [57]:
print(schedule)

[[0, 1.0, 'Household'], [1, 1, 'Household'], [2, 1, 'Household'], [3, 1, 'Household'], [4, 1, 'Household'], [5, 1, 'Household'], [6, 1, 'Household'], [7, 0.75, 'Household'], [7, 0.25, 'Household'], [8, 1.0, 'Household'], [9, 0.5, 'Household'], [9, 0.3333333333333333, 'Household'], [9, 0.08333333333333333, 'Household'], [9, 0.08333333333333333, 'Other'], [10, 0.08333333333333333, 'Other'], [10, 0.016666666666666666, 'Other'], [10, 0.06666666666666667, 'Grocery store'], [10, 0.03333333333333333, 'Grocery store'], [10, 0.08333333333333333, 'Grocery store'], [10, 0.25, 'Place of worship'], [10, 0.25, 'Place of worship'], [10, 0.21666666666666667, 'Place of worship'], [11, 0.03333333333333333, 'Place of worship'], [11, 0.25, 'Place of worship'], [11, 0.03333333333333333, 'Place of worship'], [11, 0.03333333333333333, 'Place of worship'], [11, 0.03333333333333333, 'Place of worship'], [11, 0.08333333333333333, 'Other Household'], [11, 0.016666666666666666, 'Other Household'], [11, 0.16666666

In [41]:
test.shape[0]

38

In [51]:
465 // 60

7